In [1]:
#from https://www.jitsejan.com/using-scrapy-in-jupyter-notebook.html
import scrapy
from scrapy.crawler import CrawlerProcess

In [2]:
import json

class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('linksResult.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item

In [3]:
import logging

class QuotesSpider(scrapy.Spider):
    name = "links"
    start_urls = [ 'https://en.wikipedia.org/wiki/Philosophy' ]
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 1}, # Used for pipeline 1
        'FEED_FORMAT':'json',                                 # Used for pipeline 2
        'FEED_URI': 'linksResult.json'                        # Used for pipeline 2
    }
    
    #Todo: fix parsing for wikipedia
    def parse(self, response):
        for link in response.css('div.mw-parser-output a'):
            linkString = str(link.css('::attr(href)').extract_first())
            if ( (linkString[0:6] == '/wiki/') & (linkString[6:11] != 'File:') ):
                yield {
                    'text': linkString,
                }

In [4]:
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'
})

process.crawl(QuotesSpider)
process.start()

2020-12-16 13:54:39 [scrapy.utils.log] INFO: Scrapy 2.4.1 started (bot: scrapybot)
2020-12-16 13:54:39 [scrapy.utils.log] INFO: Versions: lxml 4.6.2.0, libxml2 2.9.10, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.8.6 | packaged by conda-forge | (default, Nov 27 2020, 19:31:52) - [GCC 9.3.0], pyOpenSSL 20.0.0 (OpenSSL 1.1.1h  22 Sep 2020), cryptography 3.2.1, Platform Linux-4.9.184-linuxkit-x86_64-with-glibc2.10
2020-12-16 13:54:39 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-12-16 13:54:39 [scrapy.crawler] INFO: Overridden settings:
{'LOG_LEVEL': 30,
 'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2020-12-16 13:54:39 [py.warnings] WARNING: /opt/conda/lib/python3.8/site-packages/scrapy/extensions/feedexport.py:247: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter 

In [5]:
import pandas as pd
dfjl = pd.read_json('linksResult.jl', lines=True)
dfjl

,text
0,/wiki/Philosophy_(disambiguation)
1,/wiki/The_School_of_Athens
2,/wiki/Raphael
3,/wiki/Ancient_Greek_philosophy
4,/wiki/Ancient_Greek_architecture
...,...
1917,/wiki/LCCN_(identifier)
1918,/wiki/LNB_(identifier)
1919,/wiki/NARA_(identifier)
1920,/wiki/NDL_(identifier)


In [3]:
from neo4j import GraphDatabase, unit_of_work

class Neo4jDatabase:
    # Wrapper around Neo4j database to easily use Cypher in Python
    def __init__(self, uri, user=None, password=None):
        if user and password:
            self.driver = GraphDatabase.driver(uri, auth=(user, password))
        else: 
            self.driver = GraphDatabase.driver(uri)
            
    def __del__(self):
        self.driver.close()
    
    @unit_of_work(timeout=5)
    def _get_nodes(self, tx, limit):
        return [dict(record) for record in tx.run(f"MATCH (node) RETURN (node) LIMIT {limit}")]
    
    @unit_of_work(timeout=5)
    def _get_relations(self, tx, limit):
        return [dict(record) for record in tx.run(f"MATCH (root)-[relation]->(leaf) RETURN root, relation, leaf LIMIT {limit}")]
    
    @unit_of_work(timeout=5)
    def _add_nodes(self, tx, query):
        return tx.run(query)

    def get_all_nodes(self, limit=25):
        # Return all nodes in limit
        with self.driver.session() as session:
            return session.read_transaction(self._get_nodes, limit)
        
    def get_all_relations(self, limit=25):
        # Return all nodes in limit
        with self.driver.session() as session:
            return session.read_transaction(self._get_relations, limit)
            
    def add_new_page(self, root, leaves):
        # Add a new root node and create a relationship with all the leaves
        # This function create new leaves if it don't already exist
        query = "merge (%s:WikiPage {link: '%s'})\n" % (root, root)
        for leave in leaves:
            query += "merge (%s:WikiPage {link: '%s'})\n" % (leave, leave)
        for leave in leaves:
            query += "merge (%s)-[:IsIn]->(%s)\n" % (root, leave)
        with self.driver.session() as session:
            return session.write_transaction(self._add_nodes, query)


In [4]:
def test_neo4j_wrapper():
    database = Neo4jDatabase("bolt://db:7687")
    database.add_new_page("leaf2",["leaf3", "leaf9", "Philosophy"])
    database.add_new_page("Philosophy", ["leaf3", "leaf1", "leaf5"])
    print(database.get_all_nodes()) # should print all nodes in database
    print("\n")
    print(database.get_all_relations()) # should print all relations in database
test_neo4j_wrapper()

[{'node': <Node id=1 labels=frozenset({'WikiPage'}) properties={'link': 'leaf2'}>}, {'node': <Node id=2 labels=frozenset({'WikiPage'}) properties={'link': 'leaf3'}>}, {'node': <Node id=6 labels=frozenset({'WikiPage'}) properties={'link': 'leaf9'}>}, {'node': <Node id=7 labels=frozenset({'WikiPage'}) properties={'link': 'Philosophy'}>}, {'node': <Node id=8 labels=frozenset({'WikiPage'}) properties={'link': 'leaf1'}>}, {'node': <Node id=10 labels=frozenset({'WikiPage'}) properties={'link': 'leaf5'}>}]


[{'root': <Node id=1 labels=frozenset({'WikiPage'}) properties={'link': 'leaf2'}>, 'relation': <Relationship id=7 nodes=(<Node id=1 labels=frozenset({'WikiPage'}) properties={'link': 'leaf2'}>, <Node id=6 labels=frozenset({'WikiPage'}) properties={'link': 'leaf9'}>) type='IsIn' properties={}>, 'leaf': <Node id=6 labels=frozenset({'WikiPage'}) properties={'link': 'leaf9'}>}, {'root': <Node id=1 labels=frozenset({'WikiPage'}) properties={'link': 'leaf2'}>, 'relation': <Relationship id=6 nod